In [28]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
import geopandas as gpd
pd.options.display.float_format = '{:,.1f}'.format
import plotly.express as px
from pathlib import Path
import os

In [29]:
pc = "Devizes"

In [30]:
# Get current working directory
cwd = Path.cwd()

output_folder = f"{cwd}/../Jobs/"

In [31]:
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")

In [32]:
# Create rank for PC area
gdf['AreaPC'] = gdf['HECTARES']
df_pc_sorted = gdf.sort_values(by = "AreaPC", ascending=False)
df_pc_sorted['rankArea'] = list(range(1, df_pc_sorted.shape[0]+1))

In [33]:
dfpc = df_pc_sorted[df_pc_sorted.NAME == pc]
pcAreaRank =dfpc['rankArea'].item()

In [34]:
df = pd.read_csv("data/outputClippedData.csv")


df_pc = df[df.NAME == pc]

In [35]:
df_top_ten =  df[df['rank']<=10]
df_top_ten.to_csv("data/topTen.csv", index=False)

In [36]:
df_pc_renamed = df_pc.rename(columns={"groupColumnValue":df_pc.groupColumnName.iloc[0],
                      #"value":f"{df_pc.unitName.iloc[0]} ({df_pc.unit.iloc[0]})",
                      "valuePercent": "Percentage of PC",
                       "rank": "PC area rank"})

In [37]:
df_pc_renamed = df_pc_renamed.drop(columns = ["mapName","Unnamed: 0", "AreaPC","groupColumnName",  "rankPercent"])# 

In [38]:
df_pc_renamed = df_pc_renamed[['NAME', 'datasetName', df_pc.groupColumnName.iloc[0],'value','unitName', 'unit','Percentage of PC',"PC area rank",'source','mapGroup']]

In [40]:
#Create folder if not already there
if not os.path.exists(f"{output_folder}{pc}"):
    os.makedirs(f"{output_folder}{pc}")

# Write out to excel, with each grouping on a new sheet
with pd.ExcelWriter(f'{output_folder}{pc}/{pc}_outputReport.xlsx') as writer: 
    for group in df_pc_renamed.mapGroup.unique():
        mapGroup_df = df_pc_renamed[df_pc_renamed.mapGroup == group]
        
        mapGroup_df.to_excel(writer, sheet_name=group, index = False)
       